In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent)) 
from tool_monkey import MonkeyObserver, with_monkey, burst_rate_limit, progressive_rate_limit, logger, setup_default_logging

In [2]:
setup_default_logging(level=10)

Example 1: Using `bind_tools()` and `burst_rate_limit()`

In [ ]:
def bind_tools_burst_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_image_gen_tool 
    from langchain_core.tools import tool 
    observer=MonkeyObserver()
    scenario=burst_rate_limit()
    wrapped_tool=with_monkey(scenario, observer)(base_image_gen_tool)
    @tool
    def image_gen_tool(prompt:str, style:str):
        """
        Make a call to DALL-E API to generate an image based on the given prompt and style.
        Args:
            prompt (str): The text prompt describing the desired image.
            style (str): The artistic style to apply to the generated image.
        """
        return wrapped_tool(prompt=prompt, style=style)
    system_prompt = """You are a creative marketing assistant with access to an AI image generation tool. 
  When asked to create images, you should call the generate_image tool multiple times to generate each image."""

    user_prompt = """Generate 6 unique marketing images for a new artisan coffee shop. 
  Come up with creative prompts and generate each image using the photorealistic style."""
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    llm_with_tools=llm.bind_tools([image_gen_tool])
    max_iterations=10
    for i in range(max_iterations):
        ai_msg=llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        if not ai_msg.tool_calls:
            print(f"Agent loop finished: {ai_msg.content}")
            break
        for tool_call in ai_msg.tool_calls:
            try:
                result = image_gen_tool.invoke(tool_call)
                messages.append(result)
                print(f"Generated image {i+1}")
            except Exception as e:
                print(f"Rate limited: {e}")
                # Add error to messages so LLM sees it
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": f"Error: {e}"
                })
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

bind_tools_burst_example()

Ending call for base_image_gen_tool on success
Generated image 1
Ending call for base_image_gen_tool on success
Generated image 2
Ending call for base_image_gen_tool after exception
Rate limited: ʕ•͡-•ʔ Tool Monkey unleashed! ʕ•͡-•ʔ: Rate limit exceeded (burst). Retry after 5.0 seconds.
Ending call for base_image_gen_tool on success
Generated image 4
Ending call for base_image_gen_tool on success
Generated image 5
Ending call for base_image_gen_tool on success
Generated image 6
Ending call for base_image_gen_tool on success
Generated image 7
Agent loop finished: Here are the 6 unique marketing images for the new artisan coffee shop:

1. [A cozy artisan coffee shop with wooden furniture and warm lighting](https://fake-cdn.com/images/7942009920325075913.png)
2. [A steaming cup of artisan coffee with a latte art heart](https://fake-cdn.com/images/-2982640019527402376.png)
3. [A rustic chalkboard menu hanging on a brick wall in a coffee shop](https://fake-cdn.com/images/3697382546964471184

Example 2: Using `bind_tools()` and `progressive_rate_limit()`

In [ ]:
def bind_tools_progressive_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_image_gen_tool 
    from langchain_core.tools import tool 
    observer=MonkeyObserver()
    scenario=progressive_rate_limit()
    wrapped_tool=with_monkey(scenario, observer)(base_image_gen_tool)
    @tool
    def image_gen_tool(prompt:str, style:str):
        """
        Make a call to DALL-E API to generate an image based on the given prompt and style.
        Args:
            prompt (str): The text prompt describing the desired image.
            style (str): The artistic style to apply to the generated image.
        """
        return wrapped_tool(prompt=prompt, style=style)
    system_prompt = """You are a creative marketing assistant with access to an AI image generation tool. 
  When asked to create images, you should call the generate_image tool multiple times to generate each image."""

    user_prompt = """Generate 6 unique marketing images for a new artisan coffee shop. 
  Come up with creative prompts and generate each image using the photorealistic style."""
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    llm_with_tools=llm.bind_tools([image_gen_tool])
    max_iterations=10
    for i in range(max_iterations):
        ai_msg=llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        if not ai_msg.tool_calls:
            print(f"Agent loop finished: {ai_msg.content}")
            break
        for tool_call in ai_msg.tool_calls:
            try:
                result = image_gen_tool.invoke(tool_call)
                messages.append(result)
                print(f"Generated image {i+1}")
            except Exception as e:
                print(f"Rate limited: {e}")
                # Add error to messages so LLM sees it
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": f"Error: {e}"
                })
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

bind_tools_progressive_example()